In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD

train_df = pd.read_csv("./input/train.csv")
test_df = pd.read_csv("./input/test.csv")

# replace
# train_df['text'] = train_df['text'].str.replace('[^a-zA-Z0-9]', ' ')
# test_df['text'] =test_df['text'].str.replace('[^a-zA-Z0-9]', ' ')

## Number of words in the text ##
train_df["num_words"] = train_df["text"].apply(lambda x: len(str(x).split()))
test_df["num_words"] = test_df["text"].apply(lambda x: len(str(x).split()))

## Number of unique words in the text ##
train_df["num_unique_words"] = train_df["text"].apply(lambda x: len(set(str(x).split())))
test_df["num_unique_words"] = test_df["text"].apply(lambda x: len(set(str(x).split())))

## Number of characters in the text ##
train_df["num_chars"] = train_df["text"].apply(lambda x: len(str(x)))
test_df["num_chars"] = test_df["text"].apply(lambda x: len(str(x)))

## Number of stopwords in the text ##
eng_stopwords = [
    "a", "about", "above", "across", "after", "afterwards", "again", "against",
    "all", "almost", "alone", "along", "already", "also", "although", "always",
    "am", "among", "amongst", "amoungst", "amount", "an", "and", "another",
    "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are",
    "around", "as", "at", "back", "be", "became", "because", "become",
    "becomes", "becoming", "been", "before", "beforehand", "behind", "being",
    "below", "beside", "besides", "between", "beyond", "bill", "both",
    "bottom", "but", "by", "call", "can", "cannot", "cant", "co", "con",
    "could", "couldnt", "cry", "de", "describe", "detail", "do", "done",
    "down", "due", "during", "each", "eg", "eight", "either", "eleven", "else",
    "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone",
    "everything", "everywhere", "except", "few", "fifteen", "fifty", "fill",
    "find", "fire", "first", "five", "for", "former", "formerly", "forty",
    "found", "four", "from", "front", "full", "further", "get", "give", "go",
    "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter",
    "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his",
    "how", "however", "hundred", "i", "ie", "if", "in", "inc", "indeed",
    "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter",
    "latterly", "least", "less", "ltd", "made", "many", "may", "me",
    "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly",
    "move", "much", "must", "my", "myself", "name", "namely", "neither",
    "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone",
    "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on",
    "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our",
    "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps",
    "please", "put", "rather", "re", "same", "see", "seem", "seemed",
    "seeming", "seems", "serious", "several", "she", "should", "show", "side",
    "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone",
    "something", "sometime", "sometimes", "somewhere", "still", "such",
    "system", "take", "ten", "than", "that", "the", "their", "them",
    "themselves", "then", "thence", "there", "thereafter", "thereby",
    "therefore", "therein", "thereupon", "these", "they", "thick", "thin",
    "third", "this", "those", "though", "three", "through", "throughout",
    "thru", "thus", "to", "together", "too", "top", "toward", "towards",
    "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us",
    "very", "via", "was", "we", "well", "were", "what", "whatever", "when",
    "whence", "whenever", "where", "whereafter", "whereas", "whereby",
    "wherein", "whereupon", "wherever", "whether", "which", "while", "whither",
    "who", "whoever", "whole", "whom", "whose", "why", "will", "with",
    "within", "without", "would", "yet", "you", "your", "yours", "yourself",
    "yourselves"]
train_df["num_stopwords"] = train_df["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
test_df["num_stopwords"] = test_df["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))

## Number of punctuations in the text ##
import string
train_df["num_punctuations"] =train_df['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )
test_df["num_punctuations"] =test_df['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )

## Number of title case words in the text ##
train_df["num_words_upper"] = train_df["text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
test_df["num_words_upper"] = test_df["text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

## Number of title case words in the text ##
train_df["num_words_title"] = train_df["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
test_df["num_words_title"] = test_df["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))

## Average length of the words in the text ##
train_df["mean_word_len"] = train_df["text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test_df["mean_word_len"] = test_df["text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

In [2]:
train_df.head()

,id,text,author,num_words,num_unique_words,num_chars,num_stopwords,num_punctuations,num_words_upper,num_words_title,mean_word_len
0,id26305,"This process, however, afforded me no means of...",EAP,41,35,231,23,7,2,3,4.658537
1,id17569,It never once occurred to me that the fumbling...,HPL,14,14,71,10,1,0,1,4.142857
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,36,32,200,16,5,0,1,4.583333
3,id27763,How lovely is spring As we looked from Windsor...,MWS,34,32,206,14,4,0,4,5.088235
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,27,25,174,13,4,0,2,5.481481


In [22]:
## Prepare the data for modeling ###
author_mapping_dict = {'EAP':0, 'HPL':1, 'MWS':2}
train_y = train_df['author'].map(author_mapping_dict)
train_id = train_df['id'].values
test_id = test_df['id'].values

## add tfidf and svd
tfidf_vec = TfidfVectorizer(ngram_range=(1,3), max_df=0.8,lowercase=False, sublinear_tf=True)
full_tfidf = tfidf_vec.fit_transform(train_df['text'].values.tolist() + test_df['text'].values.tolist())
train_tfidf = tfidf_vec.transform(train_df['text'].values.tolist())
test_tfidf = tfidf_vec.transform(test_df['text'].values.tolist())

print(train_tfidf.shape,test_tfidf.shape)

# svd1
n_comp = 30
svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')
svd_obj.fit(full_tfidf)
train_svd = pd.DataFrame(svd_obj.transform(train_tfidf))
test_svd = pd.DataFrame(svd_obj.transform(test_tfidf))
print(train_svd.shape,test_svd.shape)

## add tfidf char
tfidf_vec2 = TfidfVectorizer(ngram_range=(3,7), analyzer='char',max_df=0.8, sublinear_tf=True)
full_tfidf2 = tfidf_vec2.fit_transform(train_df['text'].values.tolist() + test_df['text'].values.tolist())
train_tfidf2 = tfidf_vec2.transform(train_df['text'].values.tolist())
test_tfidf2 = tfidf_vec2.transform(test_df['text'].values.tolist())
print(train_tfidf2.shape,test_tfidf2.shape)

## add svd2
n_comp = 30
svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')
svd_obj.fit(full_tfidf2)
train_svd2 = pd.DataFrame(svd_obj.transform(train_tfidf2))
test_svd2 = pd.DataFrame(svd_obj.transform(test_tfidf2))
print(train_svd2.shape,test_svd2.shape)



(19579, 885129) (8392, 885129)
(19579, 30) (8392, 30)
(19579, 1354551) (8392, 1354551)
(19579, 30) (8392, 30)


In [23]:
## add cnt vec
c_vec = CountVectorizer(ngram_range=(1,3),max_df=0.8,lowercase=False)
c_vec.fit(train_df['text'].values.tolist() + test_df['text'].values.tolist())
train_cvec = c_vec.transform(train_df['text'].values.tolist())
test_cvec = c_vec.transform(test_df['text'].values.tolist())
print(train_cvec.shape,test_cvec.shape)

# add cnt char
c_vec2 = CountVectorizer(ngram_range=(3,7), analyzer='char',max_df=0.8)
c_vec2.fit(train_df['text'].values.tolist() + test_df['text'].values.tolist())
train_cvec2 = c_vec2.transform(train_df['text'].values.tolist())
test_cvec2 = c_vec2.transform(test_df['text'].values.tolist())
print(train_cvec2.shape,test_cvec2.shape)

(19579, 885129) (8392, 885129)
(19579, 1354551) (8392, 1354551)


In [24]:
# add tfidf to svd
cols_to_drop = ['id', 'text']
train_X = train_df.drop(cols_to_drop+['author'], axis=1).values
test_X = test_df.drop(cols_to_drop, axis=1).values
print(train_X.shape, test_X.shape)
train_X = np.hstack([train_X,train_svd,train_svd2])
test_X = np.hstack([test_X,test_svd,test_svd2])
print(train_X.shape, test_X.shape)

(19579, 8) (8392, 8)
(19579, 68) (8392, 68)


In [25]:
# add naive feature
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss

feat_cnt = 5
train_Y = train_y

help_tfidf_train,help_tfidf_test = np.zeros((19579,3)),np.zeros((8392,3))
help_tfidf_train2,help_tfidf_test2 = np.zeros((19579,3)),np.zeros((8392,3))
help_cnt1_train,help_cnt1_test = np.zeros((19579,3)),np.zeros((8392,3))
help_cnt2_train,help_cnt2_test = np.zeros((19579,3)),np.zeros((8392,3))

kf = KFold(n_splits=feat_cnt, shuffle=True, random_state=23)
for train_index, test_index in kf.split(train_tfidf):
    # tfidf to nb
    X_train, X_test = train_tfidf[train_index], train_tfidf[test_index]
    y_train, y_test = train_Y[train_index], train_Y[test_index]
    tmp_model = MultinomialNB(alpha=0.05,fit_prior=False)
    tmp_model.fit(X_train,y_train)
    tmp_train_feat = tmp_model.predict_proba(X_test)
    tmp_test_feat = tmp_model.predict_proba(test_tfidf)
    help_tfidf_train[test_index] = tmp_train_feat
    help_tfidf_test += tmp_test_feat/feat_cnt
    
    # tfidf to nb
    X_train, X_test = train_tfidf2[train_index], train_tfidf2[test_index]
    tmp_model = MultinomialNB(0.05,fit_prior=False)
    tmp_model.fit(X_train,y_train)
    tmp_train_feat = tmp_model.predict_proba(X_test)
    tmp_test_feat = tmp_model.predict_proba(test_tfidf2)
    help_tfidf_train2[test_index] = tmp_train_feat
    help_tfidf_test2 += tmp_test_feat/feat_cnt
    
    # count vec to nb
    X_train, X_test = train_cvec[train_index], train_cvec[test_index]
    tmp_model = MultinomialNB(0.05,fit_prior=False)
    tmp_model.fit(X_train,y_train)
    tmp_train_feat = tmp_model.predict_proba(X_test)
    tmp_test_feat = tmp_model.predict_proba(test_cvec)
    help_cnt1_train[test_index] = tmp_train_feat
    help_cnt1_test += tmp_test_feat/feat_cnt
    
    # count vec2 to nb 
    X_train, X_test = train_cvec2[train_index], train_cvec2[test_index]
    tmp_model = MultinomialNB(0.05,fit_prior=False)
    tmp_model.fit(X_train,y_train)
    tmp_train_feat = tmp_model.predict_proba(X_test)
    tmp_test_feat = tmp_model.predict_proba(test_cvec2)
    help_cnt2_train[test_index] = tmp_train_feat
    help_cnt2_test += tmp_test_feat/feat_cnt
    
help_train_feat = np.round(np.hstack([help_tfidf_train,help_tfidf_train2,help_cnt1_train,help_cnt2_train]),3)
help_test_feat = np.round(np.hstack([help_tfidf_test,help_tfidf_test2,help_cnt1_test,help_cnt2_test]),3)

print(help_train_feat.shape,help_test_feat.shape)
print(help_train_feat[:5])
print(help_test_feat[:5])

(19579, 12) (8392, 12)
[[ 0.835  0.036  0.129  1.     0.     0.     1.     0.     0.     1.     0.
   0.   ]
 [ 0.457  0.31   0.233  0.715  0.227  0.058  0.811  0.167  0.022  0.933
   0.067  0.   ]
 [ 0.747  0.225  0.028  0.992  0.008  0.     1.     0.     0.     1.     0.
   0.   ]
 [ 0.029  0.069  0.902  0.     0.002  0.998  0.     0.     1.     0.     0.
   1.   ]
 [ 0.334  0.387  0.279  0.949  0.032  0.019  0.215  0.402  0.383  1.     0.
   0.   ]]
[[ 0.192  0.087  0.721  0.001  0.     0.999  0.     0.     1.     0.     0.
   1.   ]
 [ 0.753  0.166  0.081  0.999  0.     0.     1.     0.     0.     1.     0.
   0.   ]
 [ 0.048  0.895  0.056  0.006  0.993  0.     0.     1.     0.     0.     1.
   0.   ]
 [ 0.518  0.435  0.047  0.324  0.676  0.     0.677  0.323  0.     0.2    0.8
   0.   ]
 [ 0.557  0.259  0.184  0.772  0.034  0.194  0.881  0.116  0.003  0.987  0.
   0.013]]


In [7]:
# add cnn feat
from keras.layers import Embedding, CuDNNLSTM, Dense, Flatten, Dropout, LSTM
from keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn import preprocessing
from sklearn.metrics import log_loss

print('import keras done')

Using TensorFlow backend.


import keras done


In [17]:
def get_cnn_feats():
    # return train pred prob and test pred prob 
    train_pred, test_pred = np.zeros((19579,3)),np.zeros((8392,3))
    FEAT_CNT = 5
    NUM_WORDS = 30000
    N = 10
    MAX_LEN = 150
    NUM_CLASSES = 3
    MODEL_P = '/tmp/nn_model.h5'
    
    tmp_X = train_df['text']
    tmp_Y = train_df['author']
    tmp_X_test = test_df['text']
    
    tokenizer = Tokenizer(num_words=NUM_WORDS)
    tokenizer.fit_on_texts(tmp_X)

    ttrain_x = tokenizer.texts_to_sequences(tmp_X)
    ttrain_x = pad_sequences(ttrain_x, maxlen=MAX_LEN)
    
    ttest_x = tokenizer.texts_to_sequences(tmp_X_test)
    ttest_x = pad_sequences(ttest_x, maxlen=MAX_LEN)

    lb = preprocessing.LabelBinarizer()
    lb.fit(tmp_Y)

    ttrain_y = lb.transform(tmp_Y)
    kf = KFold(n_splits=FEAT_CNT, shuffle=True, random_state=233)
    for train_index, test_index in kf.split(train_tfidf):
        model = Sequential()
        model.add(Embedding(NUM_WORDS, N, input_length=MAX_LEN))
        model.add(Conv1D(16,
                         3,
                         padding='valid',
                         activation='relu',
                         strides=1))
        model.add(GlobalAveragePooling1D())
        model.add(Dense(16, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(NUM_CLASSES, activation='softmax'))

        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        #model.summary()

        model_chk = ModelCheckpoint(filepath=MODEL_P, monitor='val_loss', save_best_only=True, verbose=1)
        np.random.seed(42) # for model train
        model.fit(ttrain_x[train_index], ttrain_y[train_index], 
                  validation_split=0.1,
                  batch_size=64, epochs=10, 
                  verbose=0,
                  callbacks=[model_chk],
                  shuffle=False
                 )

        model = load_model(MODEL_P)
        # save feat
        train_pred[test_index] = model.predict(ttrain_x[test_index])
        test_pred += model.predict(ttest_x)/feat_cnt
        print('------------------')
        
    return train_pred,test_pred

print('def cnn done')
cnn_train,cnn_test = get_cnn_feats()
cnn_train = np.round(cnn_train,3)
cnn_test = np.round(cnn_test,3)
print(cnn_train[:5])
print(cnn_test[:5])

def cnn done
Epoch 00001: val_loss improved from inf to 1.04320, saving model to /tmp/nn_model.h5
Epoch 00002: val_loss improved from 1.04320 to 0.79774, saving model to /tmp/nn_model.h5
Epoch 00003: val_loss improved from 0.79774 to 0.71137, saving model to /tmp/nn_model.h5
Epoch 00004: val_loss improved from 0.71137 to 0.68705, saving model to /tmp/nn_model.h5
Epoch 00005: val_loss improved from 0.68705 to 0.65715, saving model to /tmp/nn_model.h5
Epoch 00006: val_loss improved from 0.65715 to 0.61462, saving model to /tmp/nn_model.h5
Epoch 00007: val_loss improved from 0.61462 to 0.57192, saving model to /tmp/nn_model.h5
Epoch 00008: val_loss improved from 0.57192 to 0.54677, saving model to /tmp/nn_model.h5
Epoch 00009: val_loss improved from 0.54677 to 0.54507, saving model to /tmp/nn_model.h5
Epoch 00010: val_loss did not improve
Epoch 00001: val_loss improved from inf to 1.04943, saving model to /tmp/nn_model.h5
Epoch 00002: val_loss improved from 1.04943 to 0.77107, saving mode

In [9]:
# add lstm feat
def get_lstm_feats():
    # return train pred prob and test pred prob 
    train_pred, test_pred = np.zeros((19579,3)),np.zeros((8392,3))
    FEAT_CNT = 5
    NUM_WORDS = 30000
    N = 10
    MAX_LEN = 150
    NUM_CLASSES = 3
    MODEL_P = '/tmp/nn_model.h5'
    
    tmp_X = train_df['text']
    tmp_Y = train_df['author']
    tmp_X_test = test_df['text']
    
    tokenizer = Tokenizer(num_words=NUM_WORDS)
    tokenizer.fit_on_texts(tmp_X)

    ttrain_x = tokenizer.texts_to_sequences(tmp_X)
    ttrain_x = pad_sequences(ttrain_x, maxlen=MAX_LEN)
    
    ttest_x = tokenizer.texts_to_sequences(tmp_X_test)
    ttest_x = pad_sequences(ttest_x, maxlen=MAX_LEN)

    lb = preprocessing.LabelBinarizer()
    lb.fit(tmp_Y)

    ttrain_y = lb.transform(tmp_Y)
    kf = KFold(n_splits=FEAT_CNT, shuffle=True, random_state=2333)
    for train_index, test_index in kf.split(train_tfidf):
        model = Sequential()
        model.add(Embedding(NUM_WORDS, N, input_length=MAX_LEN))
        model.add(LSTM(N, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
        #model.add(CuDNNLSTM(N,return_sequences=True))
        model.add(Flatten())
        model.add(Dropout(0.2))
        model.add(Dense(NUM_CLASSES, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        #model.summary()

        model_chk = ModelCheckpoint(filepath=MODEL_P, monitor='val_loss', save_best_only=True, verbose=1)
        np.random.seed(42) # for model train
        model.fit(ttrain_x[train_index], ttrain_y[train_index], 
                  validation_split=0.1,
                  batch_size=64, epochs=5, 
                  verbose=0,
                  callbacks=[model_chk],
                  shuffle=False
                 )

        model = load_model(MODEL_P)
        # save feat
        train_pred[test_index] = model.predict(ttrain_x[test_index])
        test_pred += model.predict(ttest_x)/feat_cnt
        print('------------------')
        
    return train_pred,test_pred

print('def lstm done')
lstm_train,lstm_test = get_lstm_feats()
lstm_train = np.round(lstm_train,3)
lstm_test = np.round(lstm_test,3)
print(lstm_train[:5])
print(lstm_test[:5])

def lstm done
Epoch 00001: val_loss improved from inf to 0.82915, saving model to /tmp/nn_model.h5
Epoch 00002: val_loss improved from 0.82915 to 0.62665, saving model to /tmp/nn_model.h5
Epoch 00003: val_loss improved from 0.62665 to 0.51120, saving model to /tmp/nn_model.h5
Epoch 00004: val_loss did not improve
Epoch 00005: val_loss did not improve
Epoch 00001: val_loss improved from inf to 0.67931, saving model to /tmp/nn_model.h5
Epoch 00002: val_loss improved from 0.67931 to 0.49655, saving model to /tmp/nn_model.h5
Epoch 00003: val_loss improved from 0.49655 to 0.46448, saving model to /tmp/nn_model.h5
Epoch 00004: val_loss did not improve
Epoch 00005: val_loss did not improve
Epoch 00001: val_loss improved from inf to 0.65883, saving model to /tmp/nn_model.h5
Epoch 00002: val_loss improved from 0.65883 to 0.47803, saving model to /tmp/nn_model.h5
Epoch 00003: val_loss improved from 0.47803 to 0.44786, saving model to /tmp/nn_model.h5
Epoch 00004: val_loss did not improve
Epoch 0

In [26]:
f_train_X = np.hstack([train_X,help_train_feat,cnn_train,lstm_train])
f_test_X = np.hstack([test_X,help_test_feat,cnn_test,lstm_test])
print(f_train_X.shape, f_test_X.shape)

(19579, 86) (8392, 86)


In [27]:
from sklearn.model_selection import StratifiedKFold
def cv_test(k_cnt=3, s_flag = False):
    if s_flag:
        kf = StratifiedKFold(n_splits=k_cnt, shuffle=True, random_state=42)
    else:
        kf = KFold(n_splits=k_cnt, shuffle=True, random_state=42)
    test_pred = None
    org_train_pred = None
    avg_k_score = 0
    for train_index, test_index in kf.split(f_train_X,train_Y):
        X_train, X_test = f_train_X[train_index], f_train_X[test_index]
        y_train, y_test = train_Y[train_index], train_Y[test_index]
        params = {
                'colsample_bytree': 0.7,
                'subsample': 0.8,
                'eta': 0.1,
                'max_depth': 3,
                'eval_metric':'mlogloss',
                'objective':'multi:softprob',
                'num_class':3
                }
        
        # def mat
        d_train = xgb.DMatrix(X_train, y_train)
        d_valid = xgb.DMatrix(X_test, y_test)
        d_test = xgb.DMatrix(f_test_X)
        
        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        # train model
        m = xgb.train(params, d_train, 1000, watchlist, 
                        early_stopping_rounds=50,
                        verbose_eval=100)
        
        # get res
        train_pred = m.predict(d_train)
        valid_pred = m.predict(d_valid)
        tmp_train_pred = m.predict(xgb.DMatrix(f_train_X))
        
        # cal score
        train_score = log_loss(y_train,train_pred)
        valid_score = log_loss(y_test,valid_pred)
        print('train log loss',train_score,'valid log loss',valid_score)
        avg_k_score += valid_score
        
        
        if test_pred is None:
            test_pred = m.predict(d_test)
            org_train_pred = tmp_train_pred
        else:
            test_pred += m.predict(d_test)
            org_train_pred += tmp_train_pred

    # avg
    test_pred = test_pred / k_cnt
    test_pred = np.round(test_pred,4)
    org_train_pred = org_train_pred / k_cnt
    avg_k_score = avg_k_score/k_cnt

    submiss=pd.read_csv("./input/sample_submission.csv")
    submiss['EAP']=test_pred[:,0]
    submiss['HPL']=test_pred[:,1]
    submiss['MWS']=test_pred[:,2]
    submiss.to_csv("results/xgb_res_{}.csv".format(k_cnt),index=False)
    submiss.head(5)
    
    # train log loss
    print('local average valid loss',avg_k_score)
    print('train log loss', log_loss(train_Y,org_train_pred))
    
print('def done')

def done


In [28]:
cv_test(3, True)

[0]	train-mlogloss:0.993036	valid-mlogloss:0.995346
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 50 rounds.
[100]	train-mlogloss:0.228869	valid-mlogloss:0.298382
[200]	train-mlogloss:0.182353	valid-mlogloss:0.295783
Stopping. Best iteration:
[204]	train-mlogloss:0.180595	valid-mlogloss:0.295603

train log loss 0.163078433848 valid log loss 0.296552561864
[0]	train-mlogloss:0.995848	valid-mlogloss:0.99514
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 50 rounds.
[100]	train-mlogloss:0.241766	valid-mlogloss:0.271849
[200]	train-mlogloss:0.195486	valid-mlogloss:0.265985
Stopping. Best iteration:
[231]	train-mlogloss:0.183731	valid-mlogloss:0.265108

train log loss 0.16702808401 valid log loss 0.265636695239
[0]	train-mlogloss:0.993556	valid-mlogloss:0.994657
Multiple eval metrics have been passed: 'v

In [29]:
cv_test(3, False)

[0]	train-mlogloss:0.992988	valid-mlogloss:0.994817
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 50 rounds.
[100]	train-mlogloss:0.229215	valid-mlogloss:0.297556
[200]	train-mlogloss:0.182404	valid-mlogloss:0.296828
Stopping. Best iteration:
[166]	train-mlogloss:0.196088	valid-mlogloss:0.295635

train log loss 0.176905550816 valid log loss 0.296998004456
[0]	train-mlogloss:0.995105	valid-mlogloss:0.99573
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 50 rounds.
[100]	train-mlogloss:0.236688	valid-mlogloss:0.283001
[200]	train-mlogloss:0.189566	valid-mlogloss:0.279869
Stopping. Best iteration:
[173]	train-mlogloss:0.200992	valid-mlogloss:0.278913

train log loss 0.181318739978 valid log loss 0.279880517916
[0]	train-mlogloss:0.995587	valid-mlogloss:0.994662
Multiple eval metrics have been passed: '

In [30]:
cv_test(5, True)

[0]	train-mlogloss:0.993344	valid-mlogloss:0.995987
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 50 rounds.
[100]	train-mlogloss:0.237631	valid-mlogloss:0.300001
[200]	train-mlogloss:0.195839	valid-mlogloss:0.295173
Stopping. Best iteration:
[196]	train-mlogloss:0.197322	valid-mlogloss:0.29487

train log loss 0.18053755178 valid log loss 0.295829625781
[0]	train-mlogloss:0.993888	valid-mlogloss:0.995256
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 50 rounds.
[100]	train-mlogloss:0.239652	valid-mlogloss:0.292522
[200]	train-mlogloss:0.196984	valid-mlogloss:0.289627
Stopping. Best iteration:
[205]	train-mlogloss:0.195372	valid-mlogloss:0.289553

train log loss 0.179178705455 valid log loss 0.291050927926
[0]	train-mlogloss:0.994804	valid-mlogloss:0.992874
Multiple eval metrics have been passed: 'v

In [31]:
#cv_test(10)